[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/5.classification/TransformerClassification.ipynb)

**N.B.** Once it's open on Colab, remember to save a copy (by e.g. clicking `Copy to Drive` above).

---

Thie notebook explores using transformers for document classification.  Before starting, change the runtime to GPU: Runtime > Change runtime type > Hardware accelerator: GPU (any GPU is fine).

For an intro to models in PyTorch, see [this tutorial](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html).




Download classification data for training/evaluation.

In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/main/data/convote/train.tsv
!wget https://raw.githubusercontent.com/dbamman/anlp25/main/data/convote/dev.tsv

In [ ]:
import math
import sys
import torch
from torch import nn
from collections import Counter
from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# max sequence length
max_length=256

# limit vocabulary to top N words in training data
max_vocab=10000

# batch size
batch_size=128

# size of token representations (which dictates the size of the overall model).
d_model=16


# number of epochs
num_epochs=50

print('')
print("********************************************")
print("Running on: {}".format(device))
print("********************************************")
print('')

In [ ]:
# PositionalEncoding class copied from:
# https://github.com/pytorch/examples/blob/main/word_language_model/model.py

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_length, d_model)
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)#.transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [ ]:
class TransformerClassifier(torch.nn.Module):

    def __init__(self, num_labels, d_model, nhead=2, num_encoder_layers=1, dim_feedforward=256):

        super(TransformerClassifier, self).__init__()

        self.num_labels = num_labels
        self.embedding = nn.Embedding(num_embeddings=max_vocab+2, embedding_dim=d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            dropout=0.5
        )
        self.classifier = nn.Linear(d_model, self.num_labels)
        self.pos_encoder = PositionalEncoding(d_model)

    def forward(self, x, m):

        # put data on device (e.g., gpu)
        x = x.to(device)
        m = m.to(device)

        # convert input token IDs to word embeddings
        embed = self.embedding(x)

        # add position encodings to include information about word position within the document
        embed = self.pos_encoder(embed)

        # get transformer output
        h = self.transformer.encoder(embed, src_key_padding_mask=m)

        # Represent document as average embedding of transformer output
        h = torch.mean(h, dim=1)

        # Convert document representation into output label space
        logits = self.classifier(h)

        return logits


In [ ]:
def create_vocab_and_labels(filename, max_vocab):
    # This function creates the word vocabulary (and label ids) from the training data
    # The vocab is a mapping between word types and unique word IDs

    counts = Counter()
    labels = {}
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.rstrip().split("\t")
            lab = cols[0]
            text = word_tokenize(cols[1].lower())
            for tok in text:
                counts[tok] += 1

            if lab not in labels:
                labels[lab] = len(labels)

    vocab = {"[MASK]":0, "[UNK]":1}

    for k,v in counts.most_common(max_vocab):
        vocab[k] = len(vocab)

    return vocab, labels

In [ ]:
def read_data(filename, vocab, labels, max_length, max_docs=5000):
    # Read in data from file, up to the first max_docs documents. For each document
    # read up to max_length tokens.

    x = []
    y = []
    m = []

    with open(filename, encoding="utf-8") as file:
        for idx, line in enumerate(file):
            if idx >= max_docs:
                break
            cols = line.rstrip().split("\t")
            lab = cols[0]
            text = word_tokenize(cols[1])
            text_ids = []
            for tok in text:
                if tok in vocab:
                    text_ids.append(vocab[tok])
                else:
                    text_ids.append(vocab["[UNK]"])

            text_ids = text_ids[:max_length]

            # PyTorch (and most libraries that deal with matrix operations) expects all inputs to be the same length
            # So pad each document with 0s up to max_length
            # But keep track of the true number of tokens in the document with the "mask" list.

            # True tokens have a mask value of 0
            mask = [0] * len(text_ids)

            for i in range(len(text_ids), max_length):
                text_ids.append(vocab["[MASK]"])
                # Padded tokens have a mask value of 1
                mask.append(1)

            x.append(text_ids)
            m.append(mask)
            y.append(labels[lab])

    return x, y, m

In [ ]:
def get_batches(x, y, m, batch_size):

    # Create minibatches from the full dataset

    batches_x = []
    batches_y = []
    batches_m = []
    for i in range(0, len(x), batch_size):
        xbatch = x[i:i+batch_size]
        ybatch = y[i:i+batch_size]
        mbatch = m[i:i+batch_size]

        batches_x.append(torch.LongTensor(xbatch))
        batches_y.append(torch.LongTensor(ybatch))
        batches_m.append(torch.BoolTensor(mbatch))

    return batches_x, batches_y, batches_m

In [ ]:
def evaluate(model, all_x, all_y, all_m):

    # Calculate accuracy

    model.eval()
    corr = 0.
    total = 0.
    with torch.no_grad():
        for x, y, m in zip(all_x, all_y, all_m):
            y_preds = model.forward(x, m)
            for idx, y_pred in enumerate(y_preds):
                prediction = torch.argmax(y_pred)
                if prediction == y[idx]:
                    corr += 1.
                total += 1
    return corr / total

In [ ]:
def train(model, model_filename, train_x, train_y, train_m, dev_x, dev_y, dev_m, shuffle=False):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    cross_entropy = nn.CrossEntropyLoss()

    # Keep track of the epoch that has the best dev accuracy
    best_dev_acc = 0.
    best_dev_epoch = None

    losses = []

    # How many epochs with no changes before we quit
    patience = 20

    dev_batches_x, dev_batches_y, dev_batches_m = get_batches(dev_x, dev_y, dev_m, batch_size=batch_size)

    for epoch in range(num_epochs):

        model.train()

        if shuffle:
            # shuffle batches
            shuffled_idx = torch.randperm(len(train_x))
            train_x = [train_x[i] for i in shuffled_idx]
            train_y = [train_y[i] for i in shuffled_idx]
            train_m = [train_m[i] for i in shuffled_idx]

        train_batches_x, train_batches_y, train_batches_m = get_batches(train_x, train_y, train_m, batch_size=batch_size)

        for x, y, m in zip(train_batches_x, train_batches_y, train_batches_m):
            # Get predictions for batch x (with mask values m)
            y_pred = model.forward(x, m)
            y = y.to(device)

            # Calculate loss as cross-entropy with true labels
            loss = cross_entropy(y_pred.view(-1, model.num_labels), y.view(-1))

            # Set all gradients to zero
            optimizer.zero_grad()

            # Calculate gradients from current loss
            loss.backward()
            losses.append(loss.item())

            # Update parameters
            optimizer.step()

        dev_accuracy = evaluate(model, dev_batches_x, dev_batches_y, dev_batches_m)
        train_accuracy = evaluate(model, train_batches_x, train_batches_y, train_batches_m)

        # we're going to save the model that performs the best on *dev* data
        if dev_accuracy > best_dev_acc:
            torch.save(model.state_dict(), model_filename)
            print("%.3f is better than %.3f, saving model ..." % (dev_accuracy, best_dev_acc))
            best_dev_acc = dev_accuracy
            best_dev_epoch = epoch

        if epoch % 1 == 0:
            print("Epoch %s, dev accuracy: %.3f, train accuracy: %.3f" % (epoch, dev_accuracy, train_accuracy))

        if epoch-best_dev_epoch > patience:
          print("%s > patience (%s), stopping..." % (epoch-best_dev_epoch, patience))
          break

    model.load_state_dict(torch.load(model_filename))
    print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))

    return losses

In [ ]:
vocab, labels = create_vocab_and_labels("train.tsv", max_vocab)
train_x, train_y, train_m = read_data("train.tsv", vocab, labels, max_length=max_length)
dev_x, dev_y, dev_m = read_data("dev.tsv", vocab, labels, max_length=max_length)

In [ ]:
classifier = TransformerClassifier(num_labels=len(labels), d_model=100, dim_feedforward=512)
classifier = classifier.to(device)

train_x_batch, train_y_batch, train_m_batch = get_batches(train_x, train_y, train_m, batch_size=batch_size)
dev_x_batch, dev_y_batch, dev_m_batch = get_batches(dev_x, dev_y, dev_m, batch_size=batch_size)

losses = train(classifier, "test.model", train_x, train_y, train_m, dev_x, dev_y, dev_m)

**Q1**. Play around with this transformer as implemented and experiment with how performance on the dev data changes as a function of `d_model`, `num_encoder_layers`, `nhead`, etc.).  Describe your experiments and report dev accuracy on them below.

**Q2**. Here is some code that plots the loss curve over training. Plot the loss curve.

In [ ]:
import seaborn as sns

def plot_loss_curve(losses):
    sns.lineplot(x=range(len(losses)), y=losses)

In [ ]:
plot_loss_curve(losses)

Do you notice any regularities in the loss? Instantiate a new classifier, but this time train with `shuffle=True`.

In [ ]:
new_classifier = TransformerClassifier(num_labels=len(labels), d_model=100, dim_feedforward=512)
new_classifier = new_classifier.to(device)

shuffled_losses = train(new_classifier, "test.model", train_x, train_y, train_m, dev_x, dev_y, dev_m, shuffle=True)

Plot the loss curve of this training run. How do the loss curves compare? **Consider:** why is it generally desirable to shuffle the training data at the beginning of each epoch when performing gradient descent?

In [ ]:
plot_loss_curve(shuffled_losses)